In [1]:
import datetime
import uuid

from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType
from pyspark.sql import SparkSession
from configs import kafka_config
import os


In [2]:

os.environ[
    'PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.5.1,org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1 pyspark-shell'


In [3]:

spark = (SparkSession.builder
         .appName("KafkaStreaming")
         .master("local[*]")
         .config("spark.sql.debug.maxToStringFields", "200")
         .config("spark.sql.columnNameLengthThreshold", "200")
         .getOrCreate())


your 131072x1 screen size is bogus. expect trouble
24/11/23 12:25:11 WARN Utils: Your hostname, DESKTOP-1G0D7LM resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/11/23 12:25:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/gamelt/spark/spark-3.5.1/assembly/target/scala-2.12/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/gamelt/.ivy2/cache
The jars for the packages stored in: /home/gamelt/.ivy2/jars
org.apache.spark#spark-streaming-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f2aada04-ee69-4954-a60d-000214bba45e;1.0
	confs: [default]
	found org.apache.spark#spark-streaming-kafka-0-10_2.12;3.5.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.1 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in local-m2-cache
	found org.xerial.snappy#snappy-java;1.1.10.3 in local-m2-cache
	found org.slf4j#slf4j-api;2.0.7 in local-m2-cache
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in local-m2-cache
	found org.apache.hadoop#hadoop-client-api;3.3.4 in local-m2-cache
	found commons-logging#commons-logging;1.1.3 in local-m2-cache
	found com.google.code.findbugs#jsr305;3.0.0 in local-m2-

In [4]:

alerts_df = spark.read.csv("../data/alerts_conditions.csv", header=True)


In [5]:

window_duration = "1 minute"
sliding_interval = "30 seconds"


In [6]:

df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_config['bootstrap_servers'][0]) \
    .option("kafka.security.protocol", "SASL_PLAINTEXT") \
    .option("kafka.sasl.mechanism", "PLAIN") \
    .option("kafka.sasl.jaas.config",
            'org.apache.kafka.common.security.plain.PlainLoginModule required username="admin" password="VawEzo1ikLtrA8Ug8THa";') \
    .option("subscribe", "building_sensors_greenmoon") \
    .option("startingOffsets", "earliest") \
    .option("maxOffsetsPerTrigger", "300") \
    .load()


In [7]:

json_schema = StructType([
    StructField("sensor_id", IntegerType(), True),
    StructField("timestamp", StringType(), True),
    StructField("temperature", IntegerType(), True),
    StructField("humidity", IntegerType(), True)
])


In [8]:

avg_stats = df.selectExpr("CAST(key AS STRING) AS key_deserialized", "CAST(value AS STRING) AS value_deserialized", "*") \
    .drop('key', 'value') \
    .withColumnRenamed("key_deserialized", "key") \
    .withColumn("value_json", from_json(col("value_deserialized"), json_schema)) \
    .withColumn("timestamp", from_unixtime(col("value_json.timestamp").cast(DoubleType())).cast("timestamp")) \
    .withWatermark("timestamp", "10 seconds") \
    .groupBy(window(col("timestamp"), window_duration, sliding_interval)) \
    .agg(
    avg("value_json.temperature").alias("t_avg"),
    avg("value_json.humidity").alias("h_avg")
) \
    .drop("topic")


In [9]:

all_alerts = avg_stats.crossJoin(alerts_df)


In [10]:

valid_alerts = all_alerts \
    .where("t_avg > temperature_min AND t_avg < temperature_max") \
    .unionAll(
    all_alerts
    .where("h_avg > humidity_min AND h_avg < humidity_max")
) \
    .withColumn("timestamp", lit(str(datetime.datetime.now()))) \
    .drop("id", "humidity_min", "humidity_max", "temperature_min", "temperature_max")


In [11]:

# Для дебагінгу. Принт проміжного резульату.
# displaying_df = valid_alerts.writeStream \
#     .trigger(processingTime='10 seconds') \
#     .outputMode("update") \
#     .format("console") \
#     .start() \
#     .awaitTermination()

uuid_udf = udf(lambda: str(uuid.uuid4()), StringType())


In [12]:

prepare_to_kafka_df = valid_alerts \
    .withColumn("key", uuid_udf()) \
    .select(
    col("key"),
    to_json(struct(col("window"),
                   col("t_avg"),
                   col("h_avg"),
                   col("code"),
                   col("message"),
                   col("timestamp"))).alias("value")
)


In [ ]:
# Для дебагінгу. Принт проміжного резульату.
displaying_df = valid_alerts.writeStream \
    .trigger(processingTime='10 seconds') \
    .outputMode("update") \
    .format("console") \
    .start() \
    .awaitTermination()

24/11/23 12:32:15 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-defcc538-3a3c-408e-804f-3ce92c905205. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/11/23 12:32:15 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/11/23 12:32:19 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


-------------------------------------------
Batch: 0
-------------------------------------------
+--------------------+-----------------+-----+----+------------+--------------------+
|              window|            t_avg|h_avg|code|     message|           timestamp|
+--------------------+-----------------+-----+----+------------+--------------------+
|{2024-11-21 18:33...|36.36666666666667| 60.1| 102|It's too wet|2024-11-23 12:32:...|
+--------------------+-----------------+-----+----+------------+--------------------+



24/11/23 12:33:24 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 67273 milliseconds


-------------------------------------------
Batch: 1
-------------------------------------------
+--------------------+-----------------+------------------+----+------------+--------------------+
|              window|            t_avg|             h_avg|code|     message|           timestamp|
+--------------------+-----------------+------------------+----+------------+--------------------+
|{2024-11-21 18:44...|             39.0|39.666666666666664| 101|It's too dry|2024-11-23 12:32:...|
|{2024-11-21 20:02...|35.78947368421053| 60.31578947368421| 102|It's too wet|2024-11-23 12:32:...|
+--------------------+-----------------+------------------+----+------------+--------------------+



24/11/23 12:33:50 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 26581 milliseconds


...

24/11/23 12:45:41 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 35252 milliseconds


-------------------------------------------
Batch: 27
-------------------------------------------
+--------------------+-----+------------------+----+-------------+--------------------+
|              window|t_avg|             h_avg|code|      message|           timestamp|
+--------------------+-----+------------------+----+-------------+--------------------+
|{2024-11-23 12:45...| 29.0|53.857142857142854| 103|It's too cold|2024-11-23 12:32:...|
+--------------------+-----+------------------+----+-------------+--------------------+



24/11/23 12:46:13 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 31972 milliseconds


-------------------------------------------
Batch: 28
-------------------------------------------
+------+-----+-----+----+-------+---------+
|window|t_avg|h_avg|code|message|timestamp|
+------+-----+-----+----+-------+---------+
+------+-----+-----+----+-------+---------+



24/11/23 12:46:44 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 30808 milliseconds


In [ ]:

# query = prepare_to_kafka_df.writeStream \
#     .trigger(processingTime='30 seconds') \
#     .outputMode("update") \
#     .format("kafka") \
#     .option("kafka.bootstrap.servers", "77.81.230.104:9092") \
#     .option("topic", "avg_alerts") \
#     .option("kafka.security.protocol", "SASL_PLAINTEXT") \
#     .option("kafka.sasl.mechanism", "PLAIN") \
#     .option("kafka.sasl.jaas.config",
#             "org.apache.kafka.common.security.plain.PlainLoginModule required username='admin' password='VawEzo1ikLtrA8Ug8THa';") \
#     .option("checkpointLocation", "/tmp/checkpoints-7") \
#     .start() \
#     .awaitTermination()


24/11/21 20:02:03 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/11/21 20:02:03 WARN StreamingQueryManager: Stopping existing streaming query [id=bd929ca2-6269-41ea-bc73-a19cee7c73a8, runId=d7a7e731-e3f3-40ab-92a4-8f989f2af06e], as a new run is being started.
24/11/21 20:02:03 ERROR WriteToDataSourceV2Exec: Data source write support MicroBatchWrite[epoch: 4, writer: org.apache.spark.sql.kafka010.KafkaStreamingWrite@626516e5] is aborting.
24/11/21 20:02:03 ERROR WriteToDataSourceV2Exec: Data source write support MicroBatchWrite[epoch: 4, writer: org.apache.spark.sql.kafka010.KafkaStreamingWrite@626516e5] aborted.
24/11/21 20:02:03 WARN TaskSetManager: Lost task 0.0 in stage 20.0 (TID 1616) (10.255.255.254 executor driver): TaskKilled (Stage cancelled: Job 16 cancelled part of cancelled job group d7a7e731-e3f3-40ab-92a4-8f989f2af06e)
24/11/21 20:02:06 WARN HDFSBackedStateStoreProvider: The state for version

KeyboardInterrupt: 

24/11/21 20:18:00 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 30000 milliseconds, but spent 55211 milliseconds
24/11/21 20:18:43 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 30000 milliseconds, but spent 42289 milliseconds
